In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
# Figure out how to import regularizers
###
from tensorflow.keras.regularizers import *
import tensorflow.keras.utils as ku 
import numpy as np 
import tensorflow as tf

In [6]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

--2020-08-23 16:30:29--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-08-23 16:30:29 (107 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [14]:
total_words

3211

In [15]:
model = Sequential()
model.add(# Your Embedding Layer
          tf.keras.layers.Embedding(total_words, output_dim=150, input_length=max_sequence_len-1)
          )
model.add(# An LSTM Layer
          tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True))
          )
model.add(# A dropout layer
          tf.keras.layers.Dropout(0.3)
          )
model.add(# Another LSTM Layer
          tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100))
          )
model.add(# A Dense Layer including regularizers
          tf.keras.layers.Dense(total_words//2, kernel_regularizer=l2(0.01), activation='relu')
          )
model.add(# A Dense Layer
          tf.keras.layers.Dense(total_words, activation='softmax')
          )
# Pick an optimizer
model.compile(# Pick a loss function and an optimizer
              loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy']
              )
print(model.summary())


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 10, 150)           481650    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 10, 200)           200800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 200)           0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 200)               240800    
_________________________________________________________________
dense_4 (Dense)              (None, 1605)              322605    
_________________________________________________________________
dense_5 (Dense)              (None, 3211)              5156866   
Total params: 6,402,721
Trainable params: 6,402,721
Non-trainable params: 0
____________________________________________

In [16]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 14s 30ms/step - loss: 6.9592 - accuracy: 0.0229
Epoch 2/100
484/484 [==============================] - 14s 30ms/step - loss: 6.5102 - accuracy: 0.0221
Epoch 3/100
484/484 [==============================] - 14s 30ms/step - loss: 6.4086 - accuracy: 0.0278
Epoch 4/100
484/484 [==============================] - 14s 30ms/step - loss: 6.2873 - accuracy: 0.0325
Epoch 5/100
484/484 [==============================] - 14s 30ms/step - loss: 6.1700 - accuracy: 0.0356
Epoch 6/100
484/484 [==============================] - 14s 30ms/step - loss: 6.0546 - accuracy: 0.0404
Epoch 7/100
484/484 [==============================] - 14s 30ms/step - loss: 5.9320 - accuracy: 0.0444
Epoch 8/100
484/484 [==============================] - 14s 30ms/step - loss: 5.8125 - accuracy: 0.0497
Epoch 9/100
484/484 [==============================] - 14s 30ms/step - loss: 5.6932 - accuracy: 0.0547
Epoch 10/100
484/484 [==============================] - 15s 30ms/step - l

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)